In [18]:
%cd grounding-dino-benchmark/

/teamspace/studios/this_studio/grounding-dino-benchmark


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [19]:
import os
from glob import glob

import pandas as pd

In [41]:
pd.options.plotting.backend = "plotly"

In [48]:
def load_dataframe() -> pd.DataFrame:
    files = glob('./results/*.csv')
    dfs = []
    schema = {
        'peak_memory': 'float',
        'latency': 'float',
        'throughput': 'float',
        'context_length': 'int',
        'batch_size': 'int',
        'compile': 'bool',
        'reduce_overhead': 'bool',
        'set_high_precision': 'bool',
        'device_name': 'string',
        'out_of_memory': 'bool',
    }
    for file in files:
        dfs.append(
            pd.read_csv(
                file,
                dtype=schema
            )
        )
    data = pd.concat(dfs, ignore_index=True)
    data["config"] = (
        data.apply(
            lambda row: f"compile: {row['compile']} - reduce_overhead: {row['reduce_overhead']} - set_high_precision: {row['set_high_precision']} - device: {row['device_name']}", 
            axis=1
        )
    )
    return data

In [56]:
data = load_dataframe()
numeric_columns = data.select_dtypes(include=['float', 'int']).columns
other_columns = data.select_dtypes(include=['bool', 'string']).columns

In [72]:
data.head()

,peak_memory,latency,throughput,context_length,batch_size,compile,reduce_overhead,set_high_precision,device_name,out_of_memory,config
0,6969.463867,152.944519,6.538319,9,1,False,False,False,NVIDIA L4,False,compile: False - reduce_overhead: False - set_...
1,6969.463867,152.972290,6.537132,9,1,False,False,False,NVIDIA L4,False,compile: False - reduce_overhead: False - set_...
2,6969.463867,152.201218,6.570250,9,1,False,False,False,NVIDIA L4,False,compile: False - reduce_overhead: False - set_...
3,6969.463867,152.969223,6.537263,9,1,False,False,False,NVIDIA L4,False,compile: False - reduce_overhead: False - set_...
4,6969.463867,152.492035,6.557720,9,1,False,False,False,NVIDIA L4,False,compile: False - reduce_overhead: False - set_...


In [69]:
(
    data
    .groupby(["config", "batch_size"], as_index=False)[numeric_columns]
    .mean()
    .plot(
        y="latency", 
        x="batch_size", 
        color="config", 
        kind="line", 
        title="Average time per configuration",
        markers="o"
    )
)

In [70]:
(
    data
    .groupby(["config", "batch_size"], as_index=False)[numeric_columns]
    .mean()
    .plot(
        y="peak_memory", 
        x="batch_size", 
        color="config", 
        kind="line", 
        title="Average time per configuration",
        markers="o"
    )
)

In [71]:
(
    data
    .groupby(["config", "batch_size"], as_index=False)[numeric_columns]
    .mean()
    .plot(
        y="throughput", 
        x="batch_size", 
        color="config", 
        kind="line", 
        title="Average time per configuration",
        markers="o"
    )
)